This notebook changes supply penalties of specified technology types in Scen_CarSegments.
Before, it was being done to Cost_data but I think that was useless as values are reset in Update Costs macro.

***MAKE SURE YOU RUN SCEN_VANTYPES_TRANSLATE... BEFORE THIS!!***

In [1]:
import pandas as pd

In [2]:
# function to return SP cols from df (scen_vantypes)
def return_SPcols(powertrain, df):
    
    cols = [s for s in df.columns.tolist() if s[0:2] == 'SP']
    SPcols = [s for s in cols if powertrain in s]
    
    return SPcols

In [3]:
# function to return interpolated values in SP dictionaries (below)
def return_interpolated_value(y, powertrain, scenario, SPdata):
    
    SPdict = SPdata[scenario][powertrain]
    
    if y in SPdict:
        x = SPdict[y]
    else:
        # if it's not the last one in the sorted list
        if sorted(list(SPdict.keys())+[y]).index(y) < len(list(SPdict.keys())+[y]) - 1:
            x = SPdict[list(SPdict.keys())[sorted(list(SPdict.keys())+[y]).index(y)-1]] + (y - list(SPdict.keys())[sorted(list(SPdict.keys())+[y]).index(y)-1])/(list(SPdict.keys())[sorted(list(SPdict.keys())+[y]).index(y)] - list(SPdict.keys())[sorted(list(SPdict.keys())+[y]).index(y)-1]) * (SPdict[list(SPdict.keys())[sorted(list(SPdict.keys())+[y]).index(y)]] - SPdict[list(SPdict.keys())[sorted(list(SPdict.keys())+[y]).index(y)-1]])
        else:
            if sorted(list(SPdict.keys())+[y]).index(y) == 0:
                x = 0
            else:
                x = SPdict[list(SPdict.keys())[-1]]
    return int(x)

In [4]:
# define desired SPs as a set of years with amounts (£) in a dictionary
# keep this!


SPdata = {'OMOKA': {'Petrol': {2012: 0, 2029: 0, 2034: 5000, 2035: 5000},  # OMOKA = elec and HFC on equal footing after 2035
          'Diesel': {2012: 0, 2029: 0, 2034: 5000, 2035: 5000},
          'SPHEV': {2012: 6200, 2015: 3600, 2029: 1800, 2034: 5000, 2035: 5000},
          'SPPHEV': {2012: 80000, 2015: 8000, 2034: 5000, 2035: 5000},
          'SPBEV': {2012: 9000, 2015: 7000, 2029: 5000, 2034: 1000, 2037: 0},
          'BioV': {2012: 100000},
          'GV': {2012: 100000000},
          'HFC': {2012: 100000000, 2029: 100000, 2034: 1000, 2037: 0}},
          'ELECTRICDREAMS': {'Petrol': {2012: 0, 2029: 0, 2034: 7500, 2035: 10000},  # ED = elec trucks favoured from 2030
          'Diesel': {2012: 0, 2029: 0, 2034: 5000, 2035: 10000},
          'SPHEV': {2012: 6200, 2015: 3600, 2029: 1800, 2034: 5000, 2035: 5000},
          'SPPHEV': {2012: 80000, 2015: 8000, 2034: 5000, 2035: 5000},
          'SPBEV': {2012: 9000, 2015: 7000, 2029: 5000, 2034: 1000, 2035: 0},
          'BioV': {2012: 100000},
          'GV': {2012: 100000000},
          'HFC': {2012: 100000000}},
          'KUJINAO': {'Petrol': {2012: 0, 2029: 0, 2034: 5000, 2040: 5000},  # KUJINAO = SP across board to slow uptake 
          'Diesel': {2012: 0, 2029: 0, 2034: 5000, 2040: 5000},
          'SPHEV': {2012: 6200, 2015: 3600, 2029: 1800, 2034: 5000, 2035: 5000},
          'SPPHEV': {2012: 80000, 2015: 8000, 2034: 5000, 2035: 5000},
          'SPBEV': {2012: 9000, 2015: 7000, 2029: 5000},
          'BioV': {2012: 100000},
          'GV': {2012: 100000000},
          'HFC': {2012: 100000000}},
          'STREETFIGHTING': {'Petrol': {2012: 0},  # SF = petrol and diesel favoured
          'Diesel': {2012: 0},
          'SPHEV': {2012: 6200, 2015: 3600, 2029: 1800, 2034: 5000, 2035: 5000},
          'SPPHEV': {2012: 80000, 2015: 8000, 2034: 5000, 2035: 5000},
          'SPBEV': {2012: 90000},
          'BioV': {2012: 100000},
          'GV': {2012: 100000000},
          'HFC': {2012: 100000000}}}

In [5]:
for scenario in SPdata:
    
    # load data
    scen_vantypes = pd.read_excel(f'./../TEAM-Kenya_data_out/scenarios/Scen_VanTypes{scenario}.xlsx')
    
    for powertrain in SPdata[scenario]:
        
        for y in range(2012, 2101):
        
            scen_vantypes.loc[scen_vantypes[scen_vantypes.Year == y].index.values[0], return_SPcols(powertrain, scen_vantypes)] = return_interpolated_value(y, powertrain, scenario, SPdata)
            
    
    scen_vantypes.to_excel(f'./../TEAM-Kenya_data_out/scenarios/Scen_VanTypes{scenario}.xlsx', index=False)